<a href="https://colab.research.google.com/github/snwnkang/CS109A_Final/blob/main/Final_Project_CS109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operational Energy
Building operational energy is the required energy to run a building once it is occupied and functioning. In contrast to embodied energy, which refers to energy consumed in the production of materials, transportation and construction of the building, operational energy is primarily concerned with the energy consumption during a building's life cycle, and typically it constitutes the majority of any building's Life Cycle Energy (LCE) use.
<br>
<br>
$\Large \frac{(kWh) \: \text{Annual Energy Consumption}}{(m^2) \: \text{Total Floor area}}$
<br>
<br>
#### Energy consuming systems include:

- Space Heating
- Space Cooling
- Ventilation
- Lighting
- Plug Loads
- Water Heating

In [36]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/snwnkang/CS109A_Final/main/data/230421_operational_novent_cleaned_32k.csv?token=GHSAT0AAAAAACJYBHYQBNQUGFTUTZ7O6QO2ZKDDCJQ")

In [6]:
df.head()

,seed,Equipment,WallType,Orientation,Efficiency,NaturalVentilation,SensibleRate,LatentRate,ThermalPerformance,ThermalMassFlip,...,PumpElectricity [KWh/yr],PumpElecPerArea [KWh/yr/m^2],LightingElectricity [KWh/yr],LightElecPerArea [KWh/yr/m^2],ApplianceElectricity [KWh/yr],ApplianceElecPerArea [KWh/yr/m^2],ApplianceGas [KWh/yr],ApplianceGasPerArea [KWh/yr/m^2],HotWater [KWh/yr],HotWaterPerArea [KWh/yr/m^2]
0,91001,1,3,348,medium,0,0.577611,0.769275,normal,0,...,0.00000,0.000000,102128.0753,48.761679,49736.87014,23.747175,0.0000,0.000000,6977.650054,3.331522
1,91003,5,3,96,medium,0,0.608100,0.700198,high,0,...,47665.60951,11.487335,202331.9820,48.761679,98536.66075,23.747175,0.0000,0.000000,13823.836010,3.331522
2,91004,4,0,182,high,0,0.743813,0.787790,high,0,...,172798.48180,13.536407,301066.7194,23.584477,724209.15370,56.731922,938873.3609,73.547939,340295.030100,26.657480
3,91005,0,0,359,high,0,0.759058,0.794812,high,1,...,0.00000,0.000000,108721.9494,11.131444,313707.18830,32.118759,0.0000,0.000000,268144.972900,27.453894
4,91008,5,1,317,high,0,0.805260,0.812712,normal,0,...,219399.64710,10.991456,510409.9023,25.570450,616842.83520,30.902514,0.0000,0.000000,51150.008010,2.562507


In [7]:
from IPython import display

In [8]:
#confirm there are no missing values
df_missingness = df.isna().sum()
display.display(df_missingness)
display.Javascript("google.colab.output.setIframeHeight('300px');")

seed                                 0
Equipment                            0
WallType                             0
Orientation                          0
Efficiency                           0
NaturalVentilation                   0
SensibleRate                         0
LatentRate                           0
ThermalPerformance                   0
ThermalMassFlip                      0
nonMassWallR [m^2-K/W]               0
MassWallR [m^2-K/W]                  0
RoofR [m^2-K/W]                      0
InteriorFloorR [m^2-K/W]             0
ExteriorFloorR [m^2-K/W]             0
WWRnorth                             0
WWRwest                              0
WWRsouth                             0
WWReast                              0
SHGC                                 0
WindowR [m^2-K/W]                    0
BuildingVolumn [m^3]                 0
TotalFloorArea [m^2]                 0
f2fHeight [m]                        0
numFloor                             0
TotalRoofArea [m^2]      

<IPython.core.display.Javascript object>

In [9]:
selected_columns = [
    'program',
    'Orientation',
    'nonMassWallR [m^2-K/W]',
    'MassWallR [m^2-K/W]',
    'RoofR [m^2-K/W]',
    'InteriorFloorR [m^2-K/W]',
    'ExteriorFloorR [m^2-K/W]',
    'WWRnorth',
    'WWRwest',
    'WWRsouth',
    'WWReast',
    'SHGC',
    'WindowR [m^2-K/W]',
    'numFloor',
    'AspectRatio',
    'WallType',
    'BuildingVolumn [m^3]',
    'TotalFacadeArea [m^2]',
    'TotalFloorArea [m^2]',
    'TotalFacadeArea [m^2]',
    'TotalRoofArea [m^2]',
    'lightingLoad [W/m^2]',
    'occupancyLoad [ppl/m^2]',
    'HotWaterLoad [L/h/m^2]',
    'AnnualCoolingElectricity [KWh/yr]',
    'CoolingElecPerArea [KWh/yr/m^2]',
    'AnnualHeatingElectricity [KWh/yr',
    'HeatingElecPerArea [KWh/yr/m^2]',
    'AnnualHeatingNaturalGas [KWh/yr]',
    'HeatingGasPerArea [KWh/yr/m^2]',
    'FanElectricity [KWh/yr]',
    'FanElecPerArea [KWh/yr/m^2]',
    'PumpElectricity [KWh/yr]',
    'PumpElecPerArea [KWh/yr/m^2]',
    'LightingElectricity [KWh/yr]',
    'LightElecPerArea [KWh/yr/m^2]',
    'ApplianceElectricity [KWh/yr]',
    'ApplianceElecPerArea [KWh/yr/m^2]',
    'ApplianceGas [KWh/yr]',
    'ApplianceGasPerArea [KWh/yr/m^2]',
    'HotWater [KWh/yr]',
    'HotWaterPerArea [KWh/yr/m^2]'
]

In [10]:
df_sel = df.loc[:, selected_columns]
df_sel.head(5)

,program,Orientation,nonMassWallR [m^2-K/W],MassWallR [m^2-K/W],RoofR [m^2-K/W],InteriorFloorR [m^2-K/W],ExteriorFloorR [m^2-K/W],WWRnorth,WWRwest,WWRsouth,...,PumpElectricity [KWh/yr],PumpElecPerArea [KWh/yr/m^2],LightingElectricity [KWh/yr],LightElecPerArea [KWh/yr/m^2],ApplianceElectricity [KWh/yr],ApplianceElecPerArea [KWh/yr/m^2],ApplianceGas [KWh/yr],ApplianceGasPerArea [KWh/yr/m^2],HotWater [KWh/yr],HotWaterPerArea [KWh/yr/m^2]
0,6,348,2.88,0.00,8.38,0.7,5.38,0.23,0.11,0.58,...,0.00000,0.000000,102128.0753,48.761679,49736.87014,23.747175,0.0000,0.000000,6977.650054,3.331522
1,6,96,3.58,0.00,10.49,0.7,6.61,0.32,0.24,0.28,...,47665.60951,11.487335,202331.9820,48.761679,98536.66075,23.747175,0.0000,0.000000,13823.836010,3.331522
2,3,182,2.21,0.00,5.52,0.7,3.43,0.38,0.37,0.56,...,172798.48180,13.536407,301066.7194,23.584477,724209.15370,56.731922,938873.3609,73.547939,340295.030100,26.657480
3,5,359,0.00,2.03,5.52,0.7,3.08,0.42,0.38,0.37,...,0.00000,0.000000,108721.9494,11.131444,313707.18830,32.118759,0.0000,0.000000,268144.972900,27.453894
4,4,317,2.02,0.00,4.40,0.7,4.68,0.90,0.90,0.90,...,219399.64710,10.991456,510409.9023,25.570450,616842.83520,30.902514,0.0000,0.000000,51150.008010,2.562507


In [11]:
df_sel.describe()

,program,Orientation,nonMassWallR [m^2-K/W],MassWallR [m^2-K/W],RoofR [m^2-K/W],InteriorFloorR [m^2-K/W],ExteriorFloorR [m^2-K/W],WWRnorth,WWRwest,WWRsouth,...,PumpElectricity [KWh/yr],PumpElecPerArea [KWh/yr/m^2],LightingElectricity [KWh/yr],LightElecPerArea [KWh/yr/m^2],ApplianceElectricity [KWh/yr],ApplianceElecPerArea [KWh/yr/m^2],ApplianceGas [KWh/yr],ApplianceGasPerArea [KWh/yr/m^2],HotWater [KWh/yr],HotWaterPerArea [KWh/yr/m^2]
count,32500.000000,32500.000000,32500.000000,32500.000000,32500.000000,3.250000e+04,32500.000000,32500.000000,32500.000000,32500.000000,...,32500.000000,32500.000000,3.250000e+04,32500.000000,3.250000e+04,32500.000000,3.250000e+04,32500.000000,32500.000000,32500.000000
mean,4.489169,179.894585,2.384207,0.321344,5.930260,7.000000e-01,4.133882,0.516276,0.516068,0.516250,...,39935.204702,6.833864,2.111848e+05,34.491958,3.146161e+05,56.094904,1.527356e+05,37.832367,78505.876139,19.993414
std,2.862389,104.237783,1.472428,0.814431,1.614038,1.110240e-16,1.772313,0.264802,0.264985,0.264917,...,60534.244312,7.899791,2.247867e+05,12.843465,3.644462e+05,60.938397,2.980765e+05,75.988763,113761.018531,36.083727
min,0.000000,0.000000,0.000000,0.000000,3.760000,7.000000e-01,0.610000,0.100000,0.100000,0.100000,...,0.000000,0.000000,4.897968e+03,11.131444,4.856892e+03,7.887766,0.000000e+00,0.000000,0.000000,0.000000
25%,2.000000,90.000000,1.500000,0.000000,4.640000,7.000000e-01,3.080000,0.290000,0.290000,0.290000,...,0.000000,0.000000,5.781578e+04,24.195756,5.067745e+04,23.747175,0.000000e+00,0.000000,3695.965385,0.965401
50%,4.000000,180.000000,2.210000,0.000000,5.520000,7.000000e-01,4.680000,0.480000,0.480000,0.480000,...,12037.373555,6.027070,1.210562e+05,33.661268,1.968612e+05,32.118759,0.000000e+00,0.000000,28652.770530,6.666019
75%,7.000000,270.000000,3.580000,0.000000,6.400000,7.000000e-01,5.560000,0.820000,0.820000,0.820000,...,55654.277815,10.104479,2.801754e+05,48.725984,4.308975e+05,56.731922,1.566767e+05,34.790597,109526.156650,26.657480
max,9.000000,360.000000,5.520000,3.790000,10.490000,7.000000e-01,6.610000,0.900000,0.900000,0.900000,...,503149.874500,52.081393,1.300000e+06,52.057494,2.540000e+06,226.922176,2.200000e+06,255.286289,798098.421500,124.023484


In [12]:
program_counts = df['program'].value_counts().sort_index()
print(program_counts)

0    3227
1    3257
2    3272
3    3261
4    3236
5    3346
6    3264
7    3261
8    3155
9    3221
Name: program, dtype: int64


In [13]:
df_sel['VolumeToFacadeRatio'] = df['BuildingVolumn [m^3]'] / df['TotalFacadeArea [m^2]']

In [14]:
df_sel.head(5)

,program,Orientation,nonMassWallR [m^2-K/W],MassWallR [m^2-K/W],RoofR [m^2-K/W],InteriorFloorR [m^2-K/W],ExteriorFloorR [m^2-K/W],WWRnorth,WWRwest,WWRsouth,...,PumpElecPerArea [KWh/yr/m^2],LightingElectricity [KWh/yr],LightElecPerArea [KWh/yr/m^2],ApplianceElectricity [KWh/yr],ApplianceElecPerArea [KWh/yr/m^2],ApplianceGas [KWh/yr],ApplianceGasPerArea [KWh/yr/m^2],HotWater [KWh/yr],HotWaterPerArea [KWh/yr/m^2],VolumeToFacadeRatio
0,6,348,2.88,0.00,8.38,0.7,5.38,0.23,0.11,0.58,...,0.000000,102128.0753,48.761679,49736.87014,23.747175,0.0000,0.000000,6977.650054,3.331522,11.441270
1,6,96,3.58,0.00,10.49,0.7,6.61,0.32,0.24,0.28,...,11.487335,202331.9820,48.761679,98536.66075,23.747175,0.0000,0.000000,13823.836010,3.331522,11.387247
2,3,182,2.21,0.00,5.52,0.7,3.43,0.38,0.37,0.56,...,13.536407,301066.7194,23.584477,724209.15370,56.731922,938873.3609,73.547939,340295.030100,26.657480,8.153950
3,5,359,0.00,2.03,5.52,0.7,3.08,0.42,0.38,0.37,...,0.000000,108721.9494,11.131444,313707.18830,32.118759,0.0000,0.000000,268144.972900,27.453894,7.132345
4,4,317,2.02,0.00,4.40,0.7,4.68,0.90,0.90,0.90,...,10.991456,510409.9023,25.570450,616842.83520,30.902514,0.0000,0.000000,51150.008010,2.562507,9.796224


In [15]:
#filter out the columns with [kWh/yr/m^2]
kwh_col = [col for col in df_sel.columns if '[KWh/yr/m^2]' in col]
kwh_col

['CoolingElecPerArea [KWh/yr/m^2]',
 'HeatingElecPerArea [KWh/yr/m^2]',
 'HeatingGasPerArea [KWh/yr/m^2]',
 'FanElecPerArea [KWh/yr/m^2]',
 'PumpElecPerArea [KWh/yr/m^2]',
 'LightElecPerArea [KWh/yr/m^2]',
 'ApplianceElecPerArea [KWh/yr/m^2]',
 'ApplianceGasPerArea [KWh/yr/m^2]',
 'HotWaterPerArea [KWh/yr/m^2]']

In [16]:
df_sel['OE'] = df_sel[kwh_col].sum(axis=1)
df_sel.head(5)

,program,Orientation,nonMassWallR [m^2-K/W],MassWallR [m^2-K/W],RoofR [m^2-K/W],InteriorFloorR [m^2-K/W],ExteriorFloorR [m^2-K/W],WWRnorth,WWRwest,WWRsouth,...,LightingElectricity [KWh/yr],LightElecPerArea [KWh/yr/m^2],ApplianceElectricity [KWh/yr],ApplianceElecPerArea [KWh/yr/m^2],ApplianceGas [KWh/yr],ApplianceGasPerArea [KWh/yr/m^2],HotWater [KWh/yr],HotWaterPerArea [KWh/yr/m^2],VolumeToFacadeRatio,OE
0,6,348,2.88,0.00,8.38,0.7,5.38,0.23,0.11,0.58,...,102128.0753,48.761679,49736.87014,23.747175,0.0000,0.000000,6977.650054,3.331522,11.441270,141.615293
1,6,96,3.58,0.00,10.49,0.7,6.61,0.32,0.24,0.28,...,202331.9820,48.761679,98536.66075,23.747175,0.0000,0.000000,13823.836010,3.331522,11.387247,127.674846
2,3,182,2.21,0.00,5.52,0.7,3.43,0.38,0.37,0.56,...,301066.7194,23.584477,724209.15370,56.731922,938873.3609,73.547939,340295.030100,26.657480,8.153950,255.768779
3,5,359,0.00,2.03,5.52,0.7,3.08,0.42,0.38,0.37,...,108721.9494,11.131444,313707.18830,32.118759,0.0000,0.000000,268144.972900,27.453894,7.132345,126.355396
4,4,317,2.02,0.00,4.40,0.7,4.68,0.90,0.90,0.90,...,510409.9023,25.570450,616842.83520,30.902514,0.0000,0.000000,51150.008010,2.562507,9.796224,124.733609


In [28]:
drop_cols = [
    'BuildingVolumn [m^3]',
    'TotalFacadeArea [m^2]',
    'TotalFloorArea [m^2]',
    'TotalFacadeArea [m^2]',
    'TotalRoofArea [m^2]',
    ]

df_sel = df_sel.drop(columns=drop_cols)

In [29]:
X = df_sel.drop(columns=['OE'])
y = df_sel['OE']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [38]:
#separate the categorical variables (PROGRAM, WALL TYPE)
X_train_program = X_train[['program']]
X_train_walltype = X_train[['WallType']]

X_test_program = X_test[['program']]
X_test_walltype = X_test[['WallType']]

X_train_to_standardize = X_train.drop(['program', 'WallType'], axis=1)
X_test_to_standardize = X_test.drop(['program', 'WallType'], axis=1)

In [39]:
StdScaler = StandardScaler()
X_train_std = StdScaler.fit_transform(X_train_to_standardize)
X_test_std = StdScaler.transform(X_test_to_standardize)

In [40]:
X_train_std_df = pd.DataFrame(X_train_std, columns=X_train_to_standardize.columns)
X_test_std_df = pd.DataFrame(X_test_std, columns=X_train_to_standardize.columns)

In [41]:
X_train_std_df

,Orientation,nonMassWallR [m^2-K/W],MassWallR [m^2-K/W],RoofR [m^2-K/W],InteriorFloorR [m^2-K/W],ExteriorFloorR [m^2-K/W],WWRnorth,WWRwest,WWRsouth,WWReast,...,PumpElecPerArea [KWh/yr/m^2],LightingElectricity [KWh/yr],LightElecPerArea [KWh/yr/m^2],ApplianceElectricity [KWh/yr],ApplianceElecPerArea [KWh/yr/m^2],ApplianceGas [KWh/yr],ApplianceGasPerArea [KWh/yr/m^2],HotWater [KWh/yr],HotWaterPerArea [KWh/yr/m^2],VolumeToFacadeRatio
0,-1.381916,0.116994,-0.392517,0.281699,1.110223e-16,0.096674,-0.891185,-1.006128,-0.177140,-0.930799,...,0.589347,-0.736324,1.116360,-0.802997,-0.532969,-0.514108,-0.499659,-0.665789,-0.464120,-0.603365
1,1.243792,-1.622734,0.159003,-1.344386,1.110223e-16,-1.987742,-0.891185,-0.326741,-0.290419,-0.779911,...,-0.312625,-0.852355,-0.060217,-0.848108,-0.762947,-0.514108,-0.499659,-0.692807,-0.556326,-0.979970
2,0.525077,1.544115,-0.392517,0.571191,1.110223e-16,0.800868,1.333876,1.333983,1.333238,1.332524,...,-0.867858,-0.457129,-0.797740,-0.766838,-0.792848,-0.514108,-0.499659,-0.692807,-0.556326,1.455198
3,-0.394879,1.646052,-0.392517,0.281699,1.110223e-16,0.800868,0.277915,-0.175766,-1.272164,0.049974,...,0.268962,-0.545547,-0.690628,-0.571986,-0.415719,-0.514108,-0.499659,-0.615473,-0.485404,0.023375
4,-0.308633,-0.148043,-0.392517,-0.802358,1.110223e-16,0.305115,1.220737,1.220752,1.219960,1.219358,...,0.307979,-0.367824,-1.815670,0.147675,-0.395789,-0.514108,-0.499659,2.074911,0.203514,-0.757718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24370,-0.078644,-0.263572,-0.392517,0.417206,1.110223e-16,0.699464,1.069886,1.069777,1.068922,1.068470,...,-0.867858,-0.226182,1.116360,-0.650044,-0.532969,-0.514108,-0.499659,-0.597050,-0.464120,-0.033197
24371,-0.155307,0.810167,-0.392517,2.800899,1.110223e-16,1.392392,-0.061501,-0.704178,-1.460962,0.238584,...,-0.867858,0.598188,-0.690628,0.278980,-0.415719,-0.514108,-0.499659,-0.389426,-0.485404,-0.426102
24372,0.410082,-0.263572,-0.392517,0.417206,1.110223e-16,-1.976475,1.220737,1.220752,1.219960,1.219358,...,-0.867858,-0.139053,0.044307,-0.193936,-0.140632,-0.379287,-0.396368,-0.393169,-0.371831,0.015089
24373,-1.123178,2.128555,-0.392517,2.800899,1.110223e-16,1.392392,-0.400917,-0.364484,-0.516975,0.012252,...,-0.867858,1.116302,0.044307,0.859051,-0.140632,-0.167140,-0.396368,0.078327,-0.371831,1.428997


In [34]:
X_test_std_df

,program,Orientation,nonMassWallR [m^2-K/W],MassWallR [m^2-K/W],RoofR [m^2-K/W],InteriorFloorR [m^2-K/W],ExteriorFloorR [m^2-K/W],WWRnorth,WWRwest,WWRsouth,...,PumpElecPerArea [KWh/yr/m^2],LightingElectricity [KWh/yr],LightElecPerArea [KWh/yr/m^2],ApplianceElectricity [KWh/yr],ApplianceElecPerArea [KWh/yr/m^2],ApplianceGas [KWh/yr],ApplianceGasPerArea [KWh/yr/m^2],HotWater [KWh/yr],HotWaterPerArea [KWh/yr/m^2],VolumeToFacadeRatio
0,-1.215901,-1.017766,1.183937,-0.392517,0.133873,1.110223e-16,0.800868,1.371589,1.371727,1.370997,...,4.697650,-0.719553,1.113579,-0.240133,2.796340,0.342473,2.854543,0.397925,2.876263,-1.289886
1,0.536330,-0.576954,0.952879,-0.392517,0.281699,1.110223e-16,0.293848,-1.042036,-1.270334,-0.705773,...,-0.867858,-0.493534,1.116360,-0.730203,-0.532969,-0.514108,-0.499659,-0.633074,-0.464120,-0.514906
2,-0.515009,0.247173,-1.622734,1.016923,-0.802358,1.110223e-16,-1.390585,-0.287778,-1.496796,-0.441456,...,-0.867858,1.709758,-0.845369,3.058683,0.007515,5.686920,0.466666,5.206860,0.181471,0.078627
3,-0.515009,0.525077,-0.025718,-0.392517,-0.802358,1.110223e-16,0.305115,-0.514056,-0.628690,0.238214,...,-0.867858,0.353403,-0.845369,1.047631,0.007533,2.513424,0.466686,2.183335,0.181471,-0.231837
4,1.237223,1.368370,-0.249980,-0.392517,-0.950184,1.110223e-16,0.305115,1.107599,1.107521,1.106681,...,-0.867858,2.669737,1.372969,0.815541,-0.277215,-0.203893,-0.421475,-0.560894,-0.529607,1.208321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8120,0.886776,-0.174473,-0.025718,-0.392517,-0.802358,1.110223e-16,0.305115,0.277915,0.277159,-0.139381,...,0.309850,0.172737,0.044307,0.067592,-0.140632,-0.326597,-0.396368,-0.276064,-0.371831,2.454929
8121,1.587669,0.160927,-1.622734,1.016923,-0.802358,1.110223e-16,-1.390585,-0.551769,-0.855153,-1.536481,...,-0.722629,0.017433,-0.797740,-0.671594,-0.792848,-0.514108,-0.499659,-0.692807,-0.556326,1.445052
8122,-0.164562,0.544242,0.823758,-0.392517,-0.734604,1.110223e-16,0.305115,0.051638,-0.704178,-1.536481,...,-0.070906,-0.019989,-0.690628,-0.180958,-0.415719,-0.514108,-0.499659,-0.511602,-0.485404,-0.249576
8123,-0.515009,-0.691949,-1.622734,0.159003,-1.344386,1.110223e-16,-1.987742,-0.476343,-1.270334,-1.007848,...,0.223651,0.258351,-0.845369,0.906867,0.007533,2.290386,0.466686,1.971450,0.181471,0.142346
